In [ ]:
import json
import pandas as pd
import time
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options


In [ ]:
#these are the links we got with the links_collection module
all_links = pd.read_csv('links.csv')

#we create a new dataframe to store the infos of each houses/appartment
col_names =  ["Locality",     
              "Type_of_property",
              "Subtype_of_property",
              "Price",
              "Type_of_sale",
              "Number_of_rooms",
              "Living_Area",
              "Fully_equipped_kitchen",
              "Furnished",
              "Open_fire",
              "Terrace",
              "Terrace_Area",
              "Garden",
              "Garden_Area",
              "Surface_of_the_land",
              "Surface_area_of_the_plot_of_land",
              "Number_of_facades",
              "Swimming_pool",
              "State_of_the_building"
              ]
df_infos  = pd.DataFrame(columns = col_names)

In [ ]:
def scrape(link, driver):
    """
    input:- link of a webpage (house, appart,...)
          - driver: a driver
    return a dictionnary with the informations of the house/appart
    """
    driver.get(link)
    
    soup = BeautifulSoup(driver.page_source, "lxml")
    result = soup.find_all('script',attrs={"type" :"text/javascript"})
    for tag in result:
        if 'window.classified' in str(tag.string):#indent??????
            window_classified = tag
            
    wcs = window_classified.string
    wcs.strip()
    # Keep only the part of the string that will be converted into a dictionary
    wcs = wcs[wcs.find("{"):wcs.rfind("}")+1]
    
    # Convert it into a dictionary through json library
    house = json.loads(wcs)
    
    """
    house_dict = {
    "Locality": house["property"]["location"]["locality"],
    "Type_of_property": house["property"]["type"],
    "Subtype_of_property": house["property"]["subtype"],
    "Price": house["price"]["mainValue"],
    "Type_of_sale": house["price"]["type"],
    "Number_of_rooms": house["property"]["bedroomCount"],
    "Living_Area": house["property"]["netHabitableSurface"],
    "Fully_equipped_kitchen": house["property"]["kitchen"]["type"],
    "Furnished": house["transaction"]["sale"]["isFurnished"],
    "Open_fire": house["property"]["fireplaceExists"],
    "Terrace": house["property"]["hasTerrace"],
    "Terrace_Area": house["property"]["terraceSurface"],
    "Garden": house["property"]["hasGarden"],
    "Garden_Area": house["property"]["gardenSurface"],
    "Surface_of_the_land": None,
    "Surface_area_of_the_plot_of_land": house["property"]["land"]["surface"],
    "Number_of_facades": house["property"]["building"]["facadeCount"],
    "Swimming_pool": house["property"]["hasSwimmingPool"],
    "State_of_the_building": house["property"]["building"]["condition"]
    }
    """
    house_dict = {}

    try:
        house_dict["Locality"] = house["property"]["location"]["postalCode"]
    except:
        house_dict["Locality"] = None

    try:
        house_dict["Type_of_property"] = house["property"]["type"]
    except:
        house_dict["Type_of_property"] = None

    try:
        house_dict["Subtype_of_property"] = house["property"]["subtype"]
    except:
        house_dict["Subtype_of_property"] = None

    try:
        house_dict["Price"] = house["price"]["mainValue"]
    except:
        house_dict["Price"] = None

    try:
        house_dict["Type_of_sale"] = house["price"]["type"]
    except:
        house_dict["Type_of_sale"] = None

    try:
        house_dict["Number_of_rooms"] = house["property"]["bedroomCount"]
    except:
        house_dict["Number_of_rooms"] = None

    try:
        house_dict["Living_Area"] = house["property"]["netHabitableSurface"]
    except:
        house_dict["Living_Area"] = None

    try:
        house_dict["Fully_equipped_kitchen"] = house["property"]["kitchen"]["type"]
    except:
        house_dict["Fully_equipped_kitchen"] = None

    try:
        house_dict["Furnished"] = house["transaction"]["sale"]["isFurnished"]
    except:
        house_dict["Furnished"] = None

    try:
        house_dict["Open_fire"] = house["property"]["fireplaceExists"]
    except:
        house_dict["Open_fire"] = None

    try:
        house_dict["Terrace"] = house["property"]["hasTerrace"]
    except:
        house_dict["Terrace"] = None

    try:
        house_dict["Terrace_Area"] = house["property"]["terraceSurface"]
    except:
        house_dict["Terrace_Area"] = None

    try:
        house_dict["Garden"] = house["property"]["hasGarden"]
    except:
        house_dict["Garden"] = None

    try:
        house_dict["Garden_Area"] = house["property"]["gardenSurface"]
    except:
        house_dict["Garden_Area"] = None

    house_dict["Surface_of_the_land"] = None

    try:
        house_dict["Surface_area_of_the_plot_of_land"] = house["property"]["land"]["surface"]
    except:
        house_dict["Surface_area_of_the_plot_of_land"] = None

    try:
        house_dict["Number_of_facades"] = house["property"]["building"]["facadeCount"]
    except:
        house_dict["Number_of_facades"] = None

    try:
        house_dict["Swimming_pool"] = house["property"]["hasSwimmingPool"]
    except:
        house_dict["Swimming_pool"] = None

    try:
        house_dict["State_of_the_building"] = house["property"]["building"]["condition"]
    except:
        house_dict["State_of_the_building"] = None
    
    return house_dict

In [ ]:
def add_dict_to_df_infos(dic):
    """
    add dic as a new line in df_infos
    """
    global df_infos
    df_infos.loc[len(df_infos)] = dic.values()

In [ ]:
def save_df(name_file):
    df_infos.to_csv(name_file)

In [ ]:
def update_dataframe(start, end):
    """
    inputs: start and end are 2 integers
    this function take links of all_links from indice 'start' to indice 'end' 
    modify the df_infos dataframe
    """
    #make sure that df_infos is empty
    global df_infos 
    df_infos = pd.DataFrame(columns = col_names)
    # initiate the driver
    #driver = Chrome()
    path = "" 
    c = Options()
    c.add_argument("--headless")  
    driver = webdriver.Firefox(options=c )
    
    try:
        #connections error, driver error, cache memory errors,... lots of it can happen, even independently of our will
        #doing a "try, exept" enable us to save the results we got anyway
        for i in range(start,end):
            print("link number:", i-start, "on", end-start)
            link = all_links.iloc[i,1]
            #skip link??   => 1550 real estate project
            house_dict = scrape(link, driver)
            #time.sleep(1.5)
            add_dict_to_df_infos(house_dict)

    except:
        #if an error occured, we keep in memory until which link we could scrape
        print("error at index",i)
        final_end = end
        end = i
    finally:
        #we save the results we got, even if an error occured
        driver.close()
        if end!=start:
            name_file = "all_info" + "Sebastien" + "_" + str(start) + "_" + str(end) + ".csv"
            save_df(name_file)
            print("save"+"all_info" + "Sebastien" + "_" + str(start) + "_" + str(end) + ".csv")
        else:
            print("discard"+"all_info" + "Sebastien" + "_" + str(start) + "_" + str(end) + ".csv")
        update_dataframe(end+1, final_end)

In [ ]:
df_infos  = pd.DataFrame(columns = col_names)
print(df_infos)

In [ ]:
update_dataframe(10000, 15000)
#update_dataframe(0, 50)